In [10]:
import random
import timeit
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests as req
import json

from pprint import pprint as p
from citipy import citipy
from datetime import datetime as dt


from config import g_api_key
from config import weather_api_key

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [11]:
def lats(size_):
    lats = []
    lats = np.random.uniform(low=-90, high=90, size=size_)
    return lats

def lngs(size_):
    lngs = []
    lngs = np.random.uniform(low=-180, high=180, size=size_)
    return lngs

lats_lngs = zip(lats(5000), lngs(5000))

coords = list(lats_lngs)
coords

[(84.10891910830495, -31.744490516892),
 (-65.73256933864025, 3.6791151823182418),
 (-41.37562733698775, 87.37785899046048),
 (-12.819351034633215, -101.947855366353),
 (-39.1939473127149, 136.29555645479024),
 (-86.0579771255716, -98.2530337539956),
 (-79.88993140405894, -102.35174282887127),
 (-70.47002345201514, 67.53065033154698),
 (28.80862092882485, 1.5174764241821208),
 (-80.85316727885092, -112.22534391649836),
 (35.20861543671762, -109.6332219591633),
 (-55.483464082993486, -144.4446277671817),
 (5.422719957423283, -118.88312528433995),
 (1.993072408692072, 55.47209057118732),
 (-70.99921029244732, -74.90103935089543),
 (-84.89126765663994, -83.84617394269694),
 (62.331656133574995, 137.43922648273468),
 (-1.862843615170533, -25.851476213731786),
 (77.8420407018443, -33.2735948431164),
 (-74.96182968732668, -99.52190792521064),
 (34.0061556528855, -83.82718530382749),
 (-7.933267917603715, 109.47918421218424),
 (53.21604521195758, 129.39891000745473),
 (-58.35512977386678, -58

In [12]:
cities = []

for coordinate in coords:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)
    
len(cities)
cities

['illoqqortoormiut',
 'hermanus',
 'busselton',
 'puerto ayora',
 'victor harbor',
 'punta arenas',
 'taolanaro',
 'adrar',
 'gallup',
 'mataura',
 'cabo san lucas',
 'victoria',
 'ushuaia',
 'dzhebariki-khaya',
 'natal',
 'tasiilaq',
 'winder',
 'gombong',
 'fevralsk',
 'onguday',
 'bambous virieux',
 'rikitea',
 'buala',
 'nueva fuerza',
 'butaritari',
 'meulaboh',
 'belyy yar',
 'gunnedah',
 'airai',
 'puta',
 'magistralnyy',
 'yellowknife',
 'kapaa',
 'new norfolk',
 'umm kaddadah',
 'grand river south east',
 'upernavik',
 'eenhana',
 'nikolskoye',
 'saint-philippe',
 'bluff',
 'maloshuyka',
 'kahului',
 'shingu',
 'zhigansk',
 'east london',
 'atuona',
 'masunga',
 'vaini',
 'baykit',
 'hualmay',
 'palmer',
 'straumen',
 'shubarshi',
 'clyde river',
 'ostrovnoy',
 'barcelos',
 'coquimbo',
 'hobart',
 'chadiza',
 'bethel',
 'viligili',
 'chagda',
 'valdivia',
 'tsihombe',
 'fuerte olimpo',
 'teya',
 'chokurdakh',
 'baiao',
 'jalapa',
 'jamestown',
 'faanui',
 'uusikaupunki',
 'man

In [13]:
# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [14]:
# Loop through all the cities in our list
for i, city in enumerate(cities):
    ### 60 calls per minute
    time.sleep(2)
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
#    print(city_url)
#    break
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        city_weather = req.get(city_url).json()
        #p(city_weather)        
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        try:
            city_rain_inches = city_weather["rain"]["1h"]
        except KeyError:
            city_rain_inches = 0
        try:
            city_snow_inches = city_weather["snow"]["1h"]
        except KeyError:
            city_snow_inches = 0
            # Append the City information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Rain (inches)": city_rain_inches,
                          "Snow (inches)": city_snow_inches,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 2 of Set 1 | hermanus
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | puerto ayora
Processing Record 5 of Set 1 | victor harbor
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 8 of Set 1 | adrar
Processing Record 9 of Set 1 | gallup
Processing Record 10 of Set 1 | mataura
Processing Record 11 of Set 1 | cabo san lucas
Processing Record 12 of Set 1 | victoria
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | dzhebariki-khaya
Processing Record 15 of Set 1 | natal
Processing Record 16 of Set 1 | tasiilaq
Processing Record 17 of Set 1 | winder
Processing Record 18 of Set 1 | gombong
Processing Record 19 of Set 1 | fevralsk
City not found. Skipping...
Processing Record 20 of Set 1 | onguday
Processing Record 21 of Set 1 | bambous virieux
Processing Record 22 of Set 1 | ri

In [15]:
len(city_data)

1335

In [16]:
# convert to data frame
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,Hermanus,-34.4187,19.2345,71.80,overcast clouds,0.00,0.00,77,100,15.35,ZA
1,Busselton,-33.6500,115.3333,64.09,broken clouds,0.00,0.00,70,84,19.51,AU
2,Puerto Ayora,-0.7393,-90.3518,85.96,overcast clouds,0.00,0.00,70,100,8.79,EC
3,Victor Harbor,-35.5667,138.6167,72.91,broken clouds,0.00,0.00,87,59,3.98,AU
4,Punta Arenas,-53.1500,-70.9167,53.71,broken clouds,0.00,0.00,43,75,32.21,CL
...,...,...,...,...,...,...,...,...,...,...,...
1330,Sarny,51.3380,26.6019,31.78,snow,0.00,0.55,95,100,20.80,UA
1331,Pandharpur,17.6667,75.3333,71.13,clear sky,0.00,0.00,26,0,2.64,IN
1332,Nkhata Bay,-11.6066,34.2907,72.41,light rain,0.38,0.00,84,47,3.53,MW
1333,Dalvik,65.9702,-18.5286,22.93,light snow,0.00,0.10,74,93,28.52,IS


In [17]:
# Create the output File (CSV)
output_data_file = "Resources/WeatherPy_Database.csv"

# Export the city_data into a csv
#city_data_df.to_csv(output_data_file, index_label="City_ID")

# Export and append to the city_data into a csv
city_data_df.to_csv(output_data_file, mode = 'a', header = None , index_label="City_ID")